In [ ]:
from IPython.display import Image

# Laboratorio 3: Reconocimiento de imágenes
# Aplicación en detección de Retinopatía Diabética
## Introducción
La retinopatía diabética es una enfermedad ocular que puede producirse en pacientes con diabetes. Se da debido a los altos niveles de azúcar en la sangre que causan daño a los vasos sanguíneos en la retina. Es general que no se presenten síntomas en las etapas tempranas. Los síntomas de la retinopatía diabética suelen afectar a ambos ojos. A medida que empeora la enfermedad, se presentan los siguientes síntomas:

* Un mayor número de moscas volantes
* Visión borrosa
* Visión que cambia de borrosa a clara
* Ver áreas en blanco u oscuras en el campo de visión
* Visión nocturna deficiente
* Notar que los colores se ven atenuados o apagados
* Pérdida la visión

#### Etapas y detección
La retinopatía se puede detectar de múltiples maneras, sin embargo existe un método a través del cual este padecimiento es detectado por inspección visual de los ojos del paciente. Los ojos, específicamente “un diferencial del ojo” se pueden observar nubosidades, que son hemorragias de los vasos sanguíneos.


In [ ]:
Image("../input/assets/assets/RetinopatiaDiabetica.jpg")

##### Tipos de retinopatía diabética
* Retinopatía diabética proliferativa: Esta es la etapa más avanzada de la enfermedad. Se produce cuando la retina comienza a desarrollar nuevos vasos sanguíneos. Esto se denomina neovascularización. Estos vasos nuevos frágiles a menudo sangran hacia el vítreo. La retinopatía diabética proliferativa es muy grave y puede hacerle perder tanto la visión central como la periférica.

* Retinopatía diabética no proliferativa: Esta es la etapa temprana de la enfermedad ocular diabética. La mayoría de las personas diabéticas padecen de este tipo de retinopatía. En esta etapa de la enfermedad, muchos vasos sanguíneos pequeños sufren pérdidas y hacen que la retina se hinche. Si  se tiene retinopatía diabética no proliferativa, la visión del paciente será borrosa

##### Problemas de detección


In [ ]:
Image('../input/assets/assets/tensorflow.PNG')

La gráfica anterior muestra que incluso oftalmólogos pueden presentar inconsistencias a la hora de diagnosticar a un paciente como con, o sin, retinopatía diabética y el grado de la misma.

## Análisis del dataset

Las imágenes son de varias dimensiones, habiendo de 1050 x 1050 px, 3216 x 2136 px, entre otros, a color por lo que se recomienda modificar las imágenes a escala de grises, o bien una transformación que permita reducir la complejidad de la red neuronal, logrando reducir el uso de recursos computacionales. 

El dataset provisto contiene 3,662 observaciones para entrenar y 1,928 para testear. Las columnas que contiene el dataset es de código - de la cual no se presentará un análisis pues no brinda información esencial - y diagnosis, esta variable se distribuye de la siguiente manera:



In [ ]:
Image('../input/assets/assets/diagnosisHist.png')

In [ ]:
Image('../input/assets/assets/props.PNG')

El código para realizar las visualizaciones anteriores puede encontrarse en el archivo DataExploration.R

Se puede observar que la mayoría de imágenes, siendo casi un 50%, en el dataset de entrenamiento son diagnosticadas como “sin retinopatía diabética”, mientras que menos de un 5.3% y 8.1% son clasificadas como “severa” y “proliferativa”, respectivamente; esto puede repercutir en las respuestas que se obtengan con la red neuronal al introducir el test set, ya que podría sesgar la información y clasificar como “sin diagnosis”.

Algunas de las imágenes para cada diagnóstico son las siguientes:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os # Para importar las imagenes
from PIL import Image # Para visualizar las imagenes
import numpy as np

In [ ]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')

In [ ]:
train_df['path'] = train_df['id_code'].map(lambda x: os.path.join('../input/aptos2019-blindness-detection/train_images/','{}.png'.format(x)))
train_df['path_gray'] = train_df['id_code'].map(lambda x: os.path.join('../input/transfrom-images/transform_images/','{}.png'.format(x)))

#### 0 - No DR

In [ ]:
no_dr = Image.open(train_df[train_df.diagnosis==0].iloc[0].path)
plt.imshow(np.asarray(no_dr))

#### 1 - Mild

In [ ]:
mild = Image.open(train_df[train_df.diagnosis==1].iloc[0].path)
plt.imshow(np.asarray(mild))

#### 2 - Moderate

In [ ]:
moderate = Image.open(train_df[train_df.diagnosis==2].iloc[0].path)
plt.imshow(np.asarray(moderate))

#### 3 - Severe

In [ ]:
sev = Image.open(train_df[train_df.diagnosis==3].iloc[0].path)
plt.imshow(np.asarray(sev))

#### 4 - Proliferative

In [ ]:
prol = Image.open(train_df[train_df.diagnosis==4].iloc[0].path)
plt.imshow(np.asarray(prol))

## Metodología
Se usarán solo las primeras 611 fotos, por cuestiones de tiempo, como acordado en clase (aunque en clase se comentó que 300 era suficiente)

In [ ]:
train_df = train_df[:610]

In [ ]:
diagnosis_labs = pd.read_csv('../input/diagnosis/diagnosis.csv')
train_df_labs = pd.merge(train_df, diagnosis_labs)
train_df_labs["diagnosis_n"].value_counts().plot(kind='bar')

Se puede observar que, ya que los diagnósticos estan distribuidos aleatoriamente en el csv original, escoger los primeros 611 no parece afectar a la distribución.

### Modificación de las imágenes

In [ ]:
# Ver dimensiones de imagen
im = Image.open(train_df['path'][1])
width, height = im.size
print(width,height) 
plt.imshow(np.asarray(im))

Se puede observar que las imagenes son bastante grandes, por lo cual será necesario para empezar, cambiar su tamaño.

Para empezar, creemos nuestros train y test set a partir de train_df. Escogeremos 65% y 35% al azar, respectivamente.

In [ ]:
train, test = np.split(train_df.sample(frac=1), [int(.65*len(train_df))])

Ahora, se debe procurar compensar la sobre-representación de los casos sin retinopatía diabética en el dataset de train, ya que esto podría ser un bias que afectaría el cross validation y el test.

Para esto, se hará un oversampling.
Para referencia, esta es la distribución de nuestro nuevo train set, al escoger una muestra del 65% al azar:

In [ ]:
train_labs = pd.merge(train, diagnosis_labs)
train_labs["diagnosis_n"].value_counts().plot(kind='bar')

Se puede observar que la distribución de los diagnósticos se sigue manteniendo.

In [ ]:
max_size = train_labs['diagnosis_n'].value_counts().max()
min_size = train_labs['diagnosis_n'].value_counts().min()

lst = [train_labs]
for class_index, group in train_labs.groupby('diagnosis'):
   lst.append(group.sample(max_size-len(group), replace=True))
oversampled_train = pd.concat(lst)

In [ ]:
oversampled_train["diagnosis_n"].value_counts().plot(kind='bar')

In [ ]:
no_dr = np.random.choice(train_labs[train_labs.diagnosis==0].index, min_size, replace=False)
mild = np.random.choice(train_labs[train_labs.diagnosis==1].index, min_size, replace=False)
moderate = np.random.choice(train_labs[train_labs.diagnosis==2].index, min_size, replace=False)
severe = np.random.choice(train_labs[train_labs.diagnosis==3].index, min_size, replace=False)
proliferative = np.random.choice(train_labs[train_labs.diagnosis==4].index, min_size, replace=False)

In [ ]:
undersample_indexes = np.concatenate([no_dr,mild,moderate,severe,proliferative])

In [ ]:
undersampled_train = train_labs.loc[undersample_indexes]
undersampled_train["diagnosis_n"].value_counts().plot(kind='bar')

## Transformaciones y Redes Neuronales
Se realizarán 2 modelos, uno con el dataset undersampled y el otro oversampled. Para empezar, cargaremos las librerías a utilizar y los datos del test.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

In [ ]:
test_images = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img(test['path_gray'].iloc[i], target_size=(256,256, 1), color_mode="grayscale")
    img = image.img_to_array(img)
    img = img/255
    test_images.append(img)
X_test = np.array(test_images)

y_test = test['diagnosis'].values
y_test = to_categorical(y_test)